In [3]:
from sentence_transformers import SentenceTransformer, util
import re

In [11]:
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import wordnet
import nltk

# Download WordNet if not already
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def expand_keyword_synonyms(keyword):
    """
    Return a set of synonyms for a keyword using WordNet.
    """
    keyword = keyword.lower().strip()
    synonyms = {keyword}
    for syn in wordnet.synsets(keyword):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower().replace("_", " "))
    return synonyms

def keyword_match_score(student_answer, keywords, operator="AND"):
    """
    Calculate keyword match score with synonym weighting and phrase-level handling.
    Exact match = 1, synonym match = 0.7
    """
    student_text = student_answer.lower()
    total_keywords = len(keywords)
    if total_keywords == 0:
        return 0.0, []

    matched_keywords = []
    score_sum = 0

    for kw in keywords:
        synonyms = expand_keyword_synonyms(kw)
        exact_match = any(kw.lower() == word for word in student_text.split())
        if exact_match or kw.lower() in student_text:
            score_sum += 1.0  # Exact match full weight
            matched_keywords.append(kw)
        elif any(syn in student_text for syn in synonyms):
            score_sum += 0.7  # Synonym partial weight
            matched_keywords.append(kw + " (synonym)")

    # Apply logical operators
    if operator == "AND":
        keyword_score = min(1.0, score_sum / total_keywords)
    elif operator == "OR":
        keyword_score = 1.0 if score_sum > 0 else 0.0
    elif operator == "XOR":
        keyword_score = 1.0 if score_sum == 1 else 0.0
    elif operator == "NAND":
        keyword_score = 1.0 if score_sum < total_keywords else 0.0
    elif operator == "NOR":
        keyword_score = 1.0 if score_sum == 0 else 0.0
    else:
        keyword_score = min(1.0, score_sum / total_keywords)

    return keyword_score, matched_keywords

def context_similarity(student_answer, model_answer):
    """
    Compute semantic similarity using SBERT embeddings.
    """
    embeddings = model.encode([student_answer, model_answer], convert_to_tensor=True)
    return util.cos_sim(embeddings[0], embeddings[1]).item()

def evaluate_answer(student_answer, model_answer, keywords, operator="AND"):
    """
    Final evaluation:
    - Keyword match 40%
    - Context similarity 60%
    """
    keyword_score, matched_keywords = keyword_match_score(student_answer, keywords, operator)
    context_score = context_similarity(student_answer, model_answer)
    final_score = (0.4 * keyword_score) + (0.6 * context_score)

    return {
        "keywords_matched": matched_keywords,
        "keyword_score": round(keyword_score, 3),
        "context_score": round(context_score, 3),
        "final_score": round(final_score, 3)
    }

# ------------------- Example Usage -------------------
student = "The processor runs instructions in a computer and is considered the mind of the system."
model_ans = "CPU is the central processing unit that executes all instructions."
keywords = ["CPU", "instructions", "brain"]

result = evaluate_answer(student, model_ans, keywords, operator="AND")
print(result)

{'keywords_matched': ['CPU (synonym)', 'instructions', 'brain (synonym)'], 'keyword_score': 0.8, 'context_score': 0.735, 'final_score': 0.761}


In [16]:
import re

def detect_negation(text):
    """
    Detects if the student's answer contains negation words.
    """
    negations = ["not", "never", "no", "none", "neither", "nor", "n't"]
    text = text.lower()
    return any(neg in text for neg in negations)

def evaluate_keywords(student_answer, keywords, operator="AND", max_marks=5):
    """
    Evaluate student answer based on keywords using logical operators.
    
    - keywords: list of strings (keywords to check)
    - operator: one of ["AND", "OR", "XOR", "NOR", "NAND"]
    - max_marks: maximum marks for this question
    """
    student_text = student_answer.lower()

    # --- 1. Keyword presence ---
    presence = [1 if kw.lower() in student_text else 0 for kw in keywords]

    # --- 2. Negation detection ---
    negation = detect_negation(student_text)

    # --- 3. Apply logical operators ---
    if operator == "AND":
        result = all(presence)
    elif operator == "OR":
        result = any(presence)
    elif operator == "XOR":
        result = sum(presence) == 1
    elif operator == "NOR":
        result = not any(presence)
    elif operator == "NAND":
        result = not all(presence)
    else:
        raise ValueError("Invalid operator")

    # --- 4. Negation override → NAND behaviour ---
    if negation:
        result = not all(presence)

    # --- 5. Assign marks ---
    score = max_marks if result else 0

    return {
        "keywords": keywords,
        "presence": presence,
        "operator": operator,
        "negation_detected": negation,
        "truth_result": result,
        "score": score
    }


# ------------------- Example Usage -------------------
student_ans = "The processor is  the mind but it executes works."
keywords = ["CPU", "brain", "tasks"]

# Try with different operators
print(evaluate_keywords(student_ans, keywords, operator="AND", max_marks=5))
print(evaluate_keywords(student_ans, keywords, operator="OR", max_marks=5))
print(evaluate_keywords(student_ans, keywords, operator="NOR", max_marks=5))
print(evaluate_keywords(student_ans, keywords, operator="NAND", max_marks=5))


{'keywords': ['CPU', 'brain', 'tasks'], 'presence': [0, 0, 0], 'operator': 'AND', 'negation_detected': False, 'truth_result': False, 'score': 0}
{'keywords': ['CPU', 'brain', 'tasks'], 'presence': [0, 0, 0], 'operator': 'OR', 'negation_detected': False, 'truth_result': False, 'score': 0}
{'keywords': ['CPU', 'brain', 'tasks'], 'presence': [0, 0, 0], 'operator': 'NOR', 'negation_detected': False, 'truth_result': True, 'score': 5}
{'keywords': ['CPU', 'brain', 'tasks'], 'presence': [0, 0, 0], 'operator': 'NAND', 'negation_detected': False, 'truth_result': True, 'score': 5}
